In [1]:
#import dependencies 
import pandas as pd 
import requests 
from sqlalchemy import create_engine
import psycopg2
import config 
import queries
from datetime import datetime
import calendar

In [74]:
# using a dates as a range is several times faster, will use ranges and optimize 
date_select = pd.date_range('2022-08-01','2022-09-01',freq='D')
#extract date from datetime and convert to stirng 
dates = [str(datetime.date(i)) for i in dates]

In [ ]:
strftime("%m/%d/%Y, %H:%M:%S")

In [76]:
dates[0] 

'2022-08-01'

In [30]:
dates[0].strftime("%Y-%M-%D")

'2022-00-08/01/22'

In [46]:
range(len(dates)-1) 

range(0, 31)

In [44]:
len(dates)

31

In [2]:
#ampersand to chain filters
#must set limit above expected data or default limt set to 1000
api_url1 = "https://data.sfgov.org/resource/imvp-dq3v.json?$limit=100000&$where=session_start_dt between '2022-08-01' and '2022-08-02'"


In [472]:
class GetData():
    #api request
     def __init__(self, endpoint_url):
        self.endpoint = endpoint_url
        self.response = requests.get(url=self.endpoint)
        self.data = self.response.json() 

In [309]:
test1 = GetData(api_url1)

requesting data
data request complete


In [310]:
len(test1.data)

65382

-------

In [362]:
for i in range(len(test1.data)):
    #test1.data[i]['time_extract'] = datetime.strptime(test1.data[0]['session_start_dt'], '%Y-%m-%dT%H:%M:%S.%f').time()

In [ ]:
for i in range(len(test1.data)):
    test1.data[i]['minute']

In [463]:
def group_time(time_string):
    time_extract = datetime.strptime(time_string,'%Y-%m-%dT%H:%M:%S.%f').time()
    hour_extract = time_extract.strftime('%H')
    minute_grouped = str(round(time_extract.minute / 5) *5) 
    time_group = datetime.strptime(hour_extract + minute_grouped, '%H%M').time()
    return time_group

In [471]:
group_time('2022-08-01T11:00:24.000')
group_time('2022-08-01T05:21:31.000')
group_time('2022-08-01T11:54:00.000')
print(group_time('2022-08-01T05:21:31.000'))
print(group_time('2022-08-01T14:15:00.000'))

05:20:00
14:15:00


In [469]:
test1.data[55:100]

[{'transmission_datetime': '232340114_4_08012022054718',
  'post_id': '614-08130',
  'street_block': 'POST ST 800',
  'payment_type': 'CASH',
  'session_start_dt': '2022-08-01T05:47:18.000',
  'session_end_dt': '2022-08-01T09:20:00.000',
  'meter_event_type': 'NS',
  'gross_paid_amt': '0.25',
  'time_extract': datetime.time(0, 22, 3)},
 {'transmission_datetime': '-6708946373_11_08012022054723',
  'post_id': '869-08220',
  'street_block': 'TERRY A FRANCOIS BLVD 800',
  'payment_type': 'CREDIT CARD',
  'session_start_dt': '2022-08-01T05:47:23.000',
  'session_end_dt': '2022-08-01T09:00:23.000',
  'meter_event_type': 'NS',
  'gross_paid_amt': '2.5',
  'time_extract': datetime.time(0, 22, 3)},
 {'transmission_datetime': '232340113_4_08012022054744',
  'post_id': '600-16340',
  'street_block': 'PACIFIC AVE 1600',
  'payment_type': 'CREDIT CARD',
  'session_start_dt': '2022-08-01T05:47:44.000',
  'session_end_dt': '2022-08-01T11:00:00.000',
  'meter_event_type': 'NS',
  'gross_paid_amt': '4.

In [ ]:
group_time

In [455]:
#--create grouped time--
#create time extract and keep it datetiem.time format to derive other vriables from 
time_extract = datetime.strptime(test1.data[23]['session_start_dt'], '%Y-%m-%dT%H:%M:%S.%f').time()
#extract hour through datetime string so it is returned as 2 character string instead of 1 digit int for ceertain hours
hour_extract = time_extract.strftime('%H')
#access minute directly so it is returned as int 
minute_grouped = str(round(time_extract.minute / 5) *5) 

In [456]:
print(datetime.strptime(hour_extract + minute_grouped, '%H%M').time())

04:50:00


-----

In [6]:
df = pd.DataFrame(test1.data)

In [401]:
df.session_start_hour.unique()

array(['0', '1', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
      dtype=object)

In [20]:
4 * 2500000

10000000

### post id

In [75]:
df.post_id.nunique()

14653

## gross paid ammount 

In [32]:
df['gross_paid_amt'] = df['gross_paid_amt'].apply(lambda x: float(x))

In [71]:
#rename to dollar ammount / round to nearest half dollar 
df['gross_rounded'] = (.50 * round(df['gross_paid_amt']/.50))

In [73]:
df.gross_rounded.nunique()

94

## session date  

In [197]:
#extract date 
df['session_start_date'] = pd.to_datetime(df['session_start_dt']).dt.date

In [ ]:
df['session_start_day'].dt.day()

In [225]:
df['session_start_year'] = pd.to_datetime(df['session_start_dt']).dt.year
df['session_start_month'] = d.to_datetime(df['session_start_dt']).dt.yea

## session time 

In [209]:
#extract time
df['session_start_time'] = pd.to_datetime(df['session_start_dt']).dt.time
#extract hour 
df['session_start_hour'] = pd.to_datetime(df['session_start_dt']).dt.hour
#extract minute
df['session_start_minute'] = pd.to_datetime(df['session_start_dt']).dt.minute
#group minutes
df['session_start_minute_group'] = 5 * round(df['session_start_minute'] / 5)

In [229]:
round((df.session_start_minute.astype(int) / 5) * 5)

0        22.0
1        22.0
2        22.0
3        22.0
4        22.0
         ... 
65377    32.0
65378    32.0
65379    51.0
65380    57.0
65381    46.0
Name: session_start_minute, Length: 65382, dtype: float64

In [243]:
print(round((6.78)/5)*5)

5


In [ ]:
(df.session_start_minute /5) * 5 )

In [239]:
round((df.session_start_minute/5) * 5)

0        22.0
1        22.0
2        22.0
3        22.0
4        22.0
         ... 
65377    32.0
65378    32.0
65379    51.0
65380    57.0
65381    46.0
Name: session_start_minute, Length: 65382, dtype: float64

In [ ]:
int(5 * round(df['session_start_minute'] / 5))

In [99]:
5 * round(8/5)

10

In [285]:
samp = datetime.strptime('243','%H%M').time()
print(samp)

02:43:00


In [159]:
df.session_start_time[0]

datetime.time(0, 22, 3)

In [257]:
df['session_start_minute_group'] = df['session_start_minute_group'].astype(int)
df['session_start_minute_group'] = df['session_start_minute_group'].astype(str)
df['session_start_hour'] = df['session_start_hour'].astype(str)

### time string 

In [288]:
df['time_string'] = df.session_start_hour+df.session_start_minute_group

In [297]:
df[['session_start_time','session_start_hour','time_string']].head()

,session_start_time,session_start_hour,time_string
0,00:22:03,0,020
1,00:22:04,0,020
2,00:22:04,0,020
3,00:22:05,0,020
4,00:22:06,0,020


In [302]:
df.time_string[0]

'020'

In [306]:
print(datetime.strptime('0022','%H%M').time())

00:22:00


In [308]:
print(datetime.strptime(df.time_string[0],'%H%M').time())

02:00:00


In [295]:
df.time_string[0]

'020'

In [ ]:
#make hours 2 digits  and focus on doing this without pandas

In [ ]:
pd.to_datetime(df['session_start_hour'] df['session_start_minute'], '%H%M')

In [260]:
df.session_start_minute_group

0        20
1        20
2        20
3        20
4        20
         ..
65377    30
65378    30
65379    50
65380    55
65381    45
Name: session_start_minute_group, Length: 65382, dtype: object

In [280]:
datetime.strptime('0013','%H%M').time()

datetime.time(0, 13)

In [262]:
datetime.strptime(df['session_start_minute_group'][0],'%M').time()

00:20:00


In [187]:
 pd.to_datetime(df['session_start_minute_group'].astype(str), format='%M').dt.time

ValueError: unconverted data remains: .0

In [178]:
df['grouped_time'] = df['session_start_minute'].astype(str)

In [179]:
df.head()

,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,gross_rounded,session_start_time,session_start_minute,grouped_time
0,-6708946392_11_08012022002203,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:03.000,2022-07-31T16:16:48.000,AT,10.69,10.5,00:22:03,22,22
1,-6708946391_11_08012022002204,854-29080,LEAVENWORTH ST 2900,CASH,2022-08-01T00:22:04.000,2022-07-31T15:00:00.000,AT,5.50,5.5,00:22:04,22,22
2,-6708946400_11_08012022002204,854-29140,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:04.000,2022-07-31T17:16:48.000,AT,4.25,4.0,00:22:04,22,22
3,-6708946399_11_08012022002205,854-29060,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:05.000,2022-07-31T16:16:48.000,AT,7.69,7.5,00:22:05,22,22
4,-6708946389_11_08012022002206,854-29120,LEAVENWORTH ST 2900,CREDIT CARD,2022-08-01T00:22:06.000,2022-07-31T16:58:36.000,AT,4.25,4.0,00:22:06,22,22
